In [ ]:
from tiled.client import from_uri
from getpass import getpass

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from pprint import pprint

In [ ]:
api_key = getpass()

In [ ]:
c = from_uri("https://aimm-staging.lbl.gov", api_key=api_key)
c

In [ ]:
list(c["NCM"]["BM"])

In [ ]:
list(c["NCM"]["BM"]["NCMA"])

In [ ]:
node = c["NCM"]["BM"]["NCMA"]["Ni-L3-pristine-1"]
node

In [ ]:
pprint(dict(node.metadata))

In [ ]:
df = node.read()
df

In [ ]:
df.plot("energy", "mu_tey", label="Ni-L3")

In [ ]:
fig, ax = plt.subplots()

for k, v in c["NCM"]["BM"]["NCMA"].items():
  if v.metadata["charge"]["cycle"] == 0 and v.metadata["element"]["edge"] == "K":
    df = v.read()
    if "mutrans" in df.columns:
      df.plot("energy", "mutrans", ax=ax, label=k)

In [ ]:
fig, ax = plt.subplots()

for k, v in c["NCM"]["BM"]["NCMA"].items():
  if v.metadata["element"]["symbol"] == "Ni" and v.metadata["element"]["edge"] == "L3":
    df = v.read()
    df.plot("energy", "mu_tey", ax=ax, label=k)
    
ax.set_xlim([845, 860])

In [ ]:
fig, ax = plt.subplots()

for k, v in c["NCM"]["BM"].items():
  if "Mn-K-pristine-1" in v:
    df = v["Mn-K-pristine-1"].read()
    df.plot("energy", "mutrans", ax=ax, label=k)

In [ ]:
# register queries

from tiled.query_registration import QueryTranslationRegistry, register
from dataclasses import dataclass
import collections
import json

@register(name="raw_mongo", overwrite=True)
@dataclass
class RawMongo:
    """
    Run a MongoDB query against a given collection.
    """

    query: str  # We cannot put a dict in a URL, so this a JSON str.

    def __init__(self, query):
        if isinstance(query, collections.abc.Mapping):
            query = json.dumps(query)
        self.query = query

In [ ]:
list(c["core"]["newville"])

In [ ]:
list(c["core"]["newville"].search(RawMongo({"measurements.element.symbol" : "Ni"})))

In [ ]:
ref = c["core"]["newville"]["Ni_metal_rt_01"]
ref

In [ ]:
fig, ax = plt.subplots()

df_ref = ref["Ni-K"].read()

df_ref["mutrans"] = np.log(df_ref["i0"] / df_ref["itrans"])

df_ref.plot("energy", "mutrans", ax=ax, label="Ni metal (newville)")

df = c["NCM"]["BM"]["NCMA"]["Ni-K-pristine-1"].read()

df.plot("energy", "mutrans", ax=ax, label="NCMA")

In [ ]:
c["NCM"].export("NCM.h5")